In [1]:
import scanpy as sc
import scarches as sca
import numpy as np

### Pbmc Kang

In [2]:
adata = sc.read('kang_pbmc.h5ad')

In [3]:
adata.obs['condition_joint'] = adata.obs['condition'].tolist()
adata.obs['condition_joint'][adata.obs['condition_joint']=='nan'] = 'control'
adata.strings_to_categoricals()

<ipython-input-3-7451970088de>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adata.obs['condition_joint'][adata.obs['condition_joint']=='nan'] = 'control'
... storing 'condition_joint' as categorical


In [4]:
intr_cvae = sca.models.TRVAE.load('q_intr_cvae_nb_alpha_kl_0_001_sd_2021', adata)


INITIALIZING NEW NETWORK..............
Encoder Architecture:
	Input Layer in, out and cond: 1972 1000 5
	Hidden Layer 1 in/out: 1000 600
	Hidden Layer 2 in/out: 600 600
	Mean/Var Layer in/out: 600 282
Decoder Architecture:
	Masked linear layer in, out and cond:  282 1972 5


In [5]:
scores = intr_cvae.latent_enrich('condition_joint', comparison="control", adata=adata)

In [6]:
for k in scores:
    print(k)
    mask = np.abs(scores[k]['bf'])> 3 # bayes vactor value to filter, can use 2.3 or even lower
    s = sum(mask)
    sort = np.argsort(np.abs(scores[k]['bf'])[mask])[::-1]
    if s > 0:
        print(adata.uns['terms'][mask][sort])
        print(scores[k]['bf'][mask][sort])

stimulated
['INTERFERON_SIGNALING' 'INTERFERON_ALPHA_BETA_SIGNALIN']
[-5.11  3.15]


In [7]:
b_mask = adata.obs['cell_type_joint'] == 'B'
b_names = adata[b_mask].obs_names.tolist()
others_names = adata[~b_mask].obs_names.tolist()

scores = intr_cvae.latent_enrich(dict(B=b_names, Others=others_names), adata=adata)

In [8]:
for k in scores:
    print(k)
    mask = np.abs(scores[k]['bf'])> 3 # bayes vactor value to filter, can use 2.3 or even lower
    s = sum(mask)
    sort = np.argsort(np.abs(scores[k]['bf'])[mask])[::-1]
    if s > 0:
        print(adata.uns['terms'][mask][sort])
        print(scores[k]['bf'][mask][sort])

B
['MHC_CLASS_II_ANTIGEN_PRESENTAT' 'SIGNALING_BY_THE_B_CELL_RECEPT']
[-4.6  -3.46]
Others
['MHC_CLASS_II_ANTIGEN_PRESENTAT' 'SIGNALING_BY_THE_B_CELL_RECEPT']
[4.66 3.6 ]


In [9]:
directions = intr_cvae._latent_directions(method="sum")
scores = intr_cvae.latent_enrich('condition_joint', comparison="control", directions=directions, adata=adata)

In [10]:
for k in scores:
    print(k)
    mask = np.abs(scores[k]['bf'])> 3 # bayes vactor value to filter, can use 2.3 or even lower
    s = sum(mask)
    sort = np.argsort(np.abs(scores[k]['bf'])[mask])[::-1]
    if s > 0:
        print(adata.uns['terms'][mask][sort])
        print(scores[k]['bf'][mask][sort])

stimulated
['INTERFERON_SIGNALING' 'INTERFERON_ALPHA_BETA_SIGNALIN']
[5.29 3.2 ]
